This script calculates the mean and standard deviation of strain and dilatation rates based on iterations produced in the previous step.

Input: iteration folders produced in the previous step. Modify variables: strainfile and tensorfile based on specific region. The user is prompted to enter the variables it and n (see below).

Output: text file that contains <lon,lat,final_strain_rate,error_strain_rate,final_dilatation_rate,error_dilatation_rate>

In [2]:
import re,os,subprocess,glob
import shutil
import numpy as np
import math
import statistics

In [3]:
localdir=os.getcwd()
# based on nomenclature given in the previous script, give the name of strain and tensor files
strainfile = "italy_d02_q03_q08_b1_3D_s1_u0_strain.dat"
tensorfile = "italy_d02_q03_q08_b1_3D_s1_u0_Dtensor_6entries.dat"
# "it" is the number of iterations generated in previous step and n is the number of gridpoints 
# (number of lines in strain file)
it = int(input("Enter number of iterations (folders): ") or "100")
n = int(input("Enter number of solutions (gridpoints): ") or "8800")
SRmag = np.zeros((n,), dtype=(float,it))
SRdil = np.zeros((n,), dtype=(float,it))


Enter number of iterations (folders): 5
Enter number of solutions (gridpoints): 9500


In [5]:
def get_num(fp):
    num = fp.split("_")[1][0:-1]
    return int(num)

In [6]:
# loop over iteration directories. mag variable corresponds to strain rate magnitude, 
#and dil variable is dilatation rate

k = 0
firstdir=""
for d in sorted(glob.glob("it*/"),key=get_num):
    i=0
    print("\n---------------------------------------------------\n")
    print("Working in directory "+d+"...\n")
    if i == 0:
        firstdir=d
    with open(localdir+"/"+d+tensorfile) as sfile:
        for lin in sfile:
            if "Drr" not in lin:
              if lin:

                Drr=float(lin.split()[0])
                Drth=float(lin.split()[1])
                Drph=float(lin.split()[2])
                Dthr=float(lin.split()[1])
                Dthth=float(lin.split()[3])
                Dthph=float(lin.split()[4])
                Dphr=float(lin.split()[2])
                Dphth=float(lin.split()[4])
                Dphph=float(lin.split()[5])
                mag=math.sqrt((Drr*Drr)+(Dthth*Dthth)+(Dphph*Dphph)+2*(Drth*Drth)+2*(Drph*Drph)+2*(Dphth*Dphth))
                dil=Drr + Dthth +  Dphph
                #print(i,k)
                SRmag[i,][k] = mag
                SRdil[i,][k] = dil
                #print(Drr,Drth,Dthth)
                i += 1
    k += 1



---------------------------------------------------

Working in directory it_0/...


---------------------------------------------------

Working in directory it_1/...


---------------------------------------------------

Working in directory it_2/...


---------------------------------------------------

Working in directory it_3/...


---------------------------------------------------

Working in directory it_4/...



In [7]:
#Estimating mean and error of strain rate and dilatation rate
j = 0
with open("error_mag_dil_strainrate_it"+str(it)+".dat","w") as outfile:
    with open(localdir+"/"+firstdir+"/"+strainfile) as strain:
        for l in strain:
            if l:
                lon = l.split()[0]
                lat = l.split()[1]
                mmag = statistics.mean(SRmag[j,])
                smag = statistics.stdev(SRmag[j,])
                mdil = statistics.mean(SRdil[j,])
                sdil = statistics.stdev(SRdil[j,])
                print(lon,lat,mmag,smag,mdil,sdil,file=outfile)
                j += 1